In [1]:
import pandas as pd

nodes_df = pd.read_csv('cifar_prunned_ranks.csv')
nodes_wide_df = nodes_df.pivot(index = 'filter_num',columns='class', values='prune_score')

classes = list(nodes_df['class'].unique())
classes.remove('overall')

In [2]:
def get_layer(filter_num, df = nodes_df, col = 'layer'):
    return df.loc[(df['filter_num'] == filter_num) & (df['class'] == df['class'].unique()[0]), col].item()


nodes_wide_df.reset_index(inplace=True)

nodes_wide_df['layer'] = nodes_wide_df['filter_num'].apply(get_layer)



layers = {}
for index, row in nodes_df[nodes_df['class'] == 'overall'].iterrows():
    if row['layer'] not in layers:
        layers[row['layer']] = []
    layers[row['layer']].append(row['filter_num'])
nodes_df.tail(10)
#nodes_wide_df['filter_num_by_layer'] = nodes_wide_df.apply(lambda row: get_layer(row['filter_num'], col='filter_num_by_layer'), axis = 1)

,filter_num,layer,filter_num_by_layer,prune_score,class
1794,154,3,37,0.034860,overall
1795,155,3,38,0.041014,overall
1796,156,3,39,0.074795,overall
1797,157,3,40,0.077561,overall
1798,158,3,41,0.048469,overall
1799,159,3,42,0.071964,overall
1800,160,3,43,0.111693,overall
1801,161,3,44,0.059062,overall
1802,162,3,45,0.186180,overall
1803,163,3,46,0.178743,overall


In [3]:
import numpy as np
from sklearn import manifold
from sklearn.metrics import euclidean_distances

def add_norm_col(df,classes=classes):
    norms = []
    norm = 0
    for index, row in df.iterrows():
        for label in classes:
            norm += row[label]**2
        norm = np.sqrt(norm)
        norms.append(norm)
    norms = np.array(norms)
    df['class_norm'] = norms

add_norm_col(nodes_wide_df)   
    
layer_similarities = {}
for layer in layers:
    layer_df = nodes_wide_df[nodes_wide_df['layer'] == layer]
    for label in classes:
        layer_df[label] = layer_df.apply(lambda row : row[label]/row['class_norm'], axis = 1)   
    layer_similarities[layer] = euclidean_distances(layer_df.iloc[:,1:-2])



layer_mds = {}
for layer in layer_similarities:
	print('layer: %s'%str(layer))
	mds = manifold.MDS(n_components=2, max_iter=3000, eps=1e-9, 
      random_state=2, dissimilarity="precomputed", n_jobs=1)
	pos = mds.fit(layer_similarities[layer]).embedding_
	layer_mds[layer] = pos

#print(layer_mds)





/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/chrishamblin/miniconda3/envs/graph_viz/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i

layer: 0
layer: 1
layer: 2
layer: 3


In [4]:
nodes_df[nodes_df['class'] == classes[0]].head(5)

,filter_num,layer,filter_num_by_layer,prune_score,class
0,0,0,0,0.191817,frog
1,1,0,1,0.121295,frog
2,2,0,2,0.244416,frog
3,3,0,3,0.034859,frog
4,4,0,4,0.050036,frog


In [5]:
import numpy as np
import igraph as ig
import json
import urllib

N = len(nodes_wide_df.index)
sizes = list(nodes_wide_df['overall'])
format_sizes = []
for size in sizes:
    format_sizes.append(50*np.cbrt(size))
sizes = format_sizes

#N=len(data_dict['nodes'])


In [14]:
Edges_full = {}
for layer in layers:
    if layer+1 in layers:
        Edges_full[layer+1] = []
        for node in layers[layer]:
            for node_2 in layers[layer+1]:
                Edges_full[layer+1].append((node,node_2))             
       
  
#print(Edges_full)

{1: [(0, 45), (0, 46), (0, 47), (0, 48), (0, 49), (0, 50), (0, 51), (0, 52), (0, 53), (0, 54), (0, 55), (0, 56), (0, 57), (0, 58), (0, 59), (0, 60), (0, 61), (0, 62), (0, 63), (0, 64), (0, 65), (0, 66), (0, 67), (1, 45), (1, 46), (1, 47), (1, 48), (1, 49), (1, 50), (1, 51), (1, 52), (1, 53), (1, 54), (1, 55), (1, 56), (1, 57), (1, 58), (1, 59), (1, 60), (1, 61), (1, 62), (1, 63), (1, 64), (1, 65), (1, 66), (1, 67), (2, 45), (2, 46), (2, 47), (2, 48), (2, 49), (2, 50), (2, 51), (2, 52), (2, 53), (2, 54), (2, 55), (2, 56), (2, 57), (2, 58), (2, 59), (2, 60), (2, 61), (2, 62), (2, 63), (2, 64), (2, 65), (2, 66), (2, 67), (3, 45), (3, 46), (3, 47), (3, 48), (3, 49), (3, 50), (3, 51), (3, 52), (3, 53), (3, 54), (3, 55), (3, 56), (3, 57), (3, 58), (3, 59), (3, 60), (3, 61), (3, 62), (3, 63), (3, 64), (3, 65), (3, 66), (3, 67), (4, 45), (4, 46), (4, 47), (4, 48), (4, 49), (4, 50), (4, 51), (4, 52), (4, 53), (4, 54), (4, 55), (4, 56), (4, 57), (4, 58), (4, 59), (4, 60), (4, 61), (4, 62), (4, 6

In [1]:
edges_test = [(0, 45), (0, 46), (0, 47), (0, 48), (0, 49), (0, 50), (0, 51), (0, 52), (0, 53), (0, 54), (0, 55), (0, 56), (0, 57), (0, 58), (0, 59), (0, 60), (0, 61), (0, 62), (0, 63), (0, 64), (0, 65), (0, 66), (0, 67), (1, 45), (1, 46), (1, 47), (1, 48), (1, 49), (1, 50), (1, 51), (1, 52), (1, 53), (1, 54), (1, 55), (1, 56), (1, 57), (1, 58), (1, 59), (1, 60), (1, 61), (1, 62), (1, 63), (1, 64), (1, 65), (1, 66), (1, 67), (2, 45), (2, 46), (2, 47), (2, 48), (2, 49), (2, 50), (2, 51), (2, 52), (2, 53), (2, 54), (2, 55), (2, 56), (2, 57), (2, 58), (2, 59), (2, 60), (2, 61), (2, 62), (2, 63), (2, 64), (2, 65), (2, 66), (2, 67), (3, 45), (3, 46), (3, 47), (3, 48), (3, 49), (3, 50), (3, 51), (3, 52), (3, 53), (3, 54), (3, 55), (3, 56), (3, 57), (3, 58), (3, 59), (3, 60), (3, 61), (3, 62), (3, 63), (3, 64), (3, 65), (3, 66), (3, 67), (4, 45), (4, 46), (4, 47), (4, 48), (4, 49), (4, 50), (4, 51), (4, 52), (4, 53), (4, 54), (4, 55), (4, 56), (4, 57), (4, 58), (4, 59), (4, 60), (4, 61), (4, 62), (4, 63), (4, 64), (4, 65), (4, 66), (4, 67), (5, 45), (5, 46), (5, 47), (5, 48), (5, 49), (5, 50), (5, 51), (5, 52), (5, 53), (5, 54), (5, 55), (5, 56), (5, 57), (5, 58), (5, 59), (5, 60), (5, 61), (5, 62), (5, 63), (5, 64), (5, 65), (5, 66), (5, 67), (6, 45), (6, 46), (6, 47), (6, 48), (6, 49), (6, 50), (6, 51), (6, 52), (6, 53), (6, 54), (6, 55), (6, 56), (6, 57), (6, 58), (6, 59), (6, 60), (6, 61), (6, 62), (6, 63), (6, 64), (6, 65), (6, 66), (6, 67), (7, 45), (7, 46), (7, 47), (7, 48), (7, 49), (7, 50), (7, 51), (7, 52), (7, 53), (7, 54), (7, 55), (7, 56), (7, 57), (7, 58), (7, 59), (7, 60), (7, 61), (7, 62), (7, 63), (7, 64), (7, 65), (7, 66), (7, 67), (8, 45), (8, 46), (8, 47), (8, 48), (8, 49), (8, 50), (8, 51), (8, 52), (8, 53), (8, 54), (8, 55), (8, 56), (8, 57), (8, 58), (8, 59), (8, 60), (8, 61), (8, 62), (8, 63), (8, 64), (8, 65), (8, 66), (8, 67), (9, 45), (9, 46), (9, 47), (9, 48), (9, 49), (9, 50), (9, 51), (9, 52), (9, 53), (9, 54), (9, 55), (9, 56), (9, 57), (9, 58), (9, 59), (9, 60), (9, 61), (9, 62), (9, 63), (9, 64), (9, 65), (9, 66), (9, 67), (10, 45), (10, 46), (10, 47), (10, 48), (10, 49), (10, 50), (10, 51), (10, 52), (10, 53), (10, 54), (10, 55), (10, 56)]
edge_test_positions = #Edge Positions
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[Xn[e[0]],Xn[e[1]], None]# x-coordinates of edge ends
    Ye+=[Yn[e[0]],Yn[e[1]], None]
    Ze+=[Zn[e[0]],Zn[e[1]], None]

In [15]:
labels_list=(list(nodes_df[nodes_df['class'] == 'overall'].filter_num))
layers_list=(list(nodes_df[nodes_df['class'] == 'overall'].layer))
N = len(labels_list)

In [7]:
nodes_df[nodes_df['class']=='overall'].iloc[179].prune_score
-(.1-1)**4+1

IndexError: single positional indexer is out-of-bounds

In [16]:
target_class = 'frog'


#Node Opacity
layer_colors = {0:'rgba(64,1,84,',
                1:'rgba(49,102,141,',
                2:'rgba(55,181,120,',
                3:'rgba(253,231,37,'}

def color_scaling(x):
    return -(x-1)**4+1

colors_list = []
for i in range(N):
    layer = nodes_df[nodes_df['class']=='overall'].iloc[i].layer
    alpha = color_scaling(nodes_df[nodes_df['class']==target_class].iloc[i].prune_score)
    colors_list.append(layer_colors[layer]+str(round(alpha,3))+')')
#print(colors_list)

#Node positions
layer_distance = 1   # distance in X direction each layer is separated by
node_positions = {}
layer_offset = 0
for layer in layer_mds:
    node_positions['layer '+str(layer_offset+1)] = {}
    node_positions['layer '+str(layer_offset+1)]['X'] = [] 
    node_positions['layer '+str(layer_offset+1)]['Y'] = [] 
    node_positions['layer '+str(layer_offset+1)]['Z'] = []  
    for i in range(len(layer_mds[layer])): 
        node_positions['layer '+str(layer_offset+1)]['Y'].append(layer_mds[layer][i][0])
        node_positions['layer '+str(layer_offset+1)]['Z'].append(layer_mds[layer][i][1])
        node_positions['layer '+str(layer_offset+1)]['X'].append(layer_offset)
    layer_offset+=1

    
#Edge selection
edge_weights = []
edge_threshold = .1
Edges = []
for i in range(len(Edges_full)):
    edge_weight = nodes_df[nodes_df['class']==target_class].iloc[Edges_full[i][0]].prune_score*nodes_df[nodes_df['class']==target_class].iloc[Edges_full[i][1]].prune_score
    if edge_weight > edge_threshold:
        Edges.append(Edges_full[i])
        edge_weights.append(edge_weight)
        
#Edge Positions

Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[Xn[e[0]],Xn[e[1]], None]# x-coordinates of edge ends
    Ye+=[Yn[e[0]],Yn[e[1]], None]
    Ze+=[Zn[e[0]],Zn[e[1]], None]    
    

'''
#node positions
Xn = []
Yn = []
Zn = []
layer_offset = 0
for layer in layer_mds:
    for i in range(len(layer_mds[layer])): 
        Yn.append(layer_mds[layer][i][0])
        Zn.append(layer_mds[layer][i][1])
        Xn.append(layer_offset)
    layer_offset+=1
    

#Edge Positions
Xe=[]
Ye=[]
Ze=[]
for e in Edges:
    Xe+=[Xn[e[0]],Xn[e[1]], None]# x-coordinates of edge ends
    Ye+=[Yn[e[0]],Yn[e[1]], None]
    Ze+=[Zn[e[0]],Zn[e[1]], None]
'''


#import chart_studio.plotly as py
import plotly.offline as py    #added
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)   #added


trace1=go.Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=dict(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )



trace2=go.Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=dict(symbol='circle',
                             size=6,
                             opacity=.99,
                             color=colors_list,
                             colorscale='Viridis',
                             line=dict(color='rgb(50,50,50)', width=0.5)
                             ),
               text=labels_list,
               hoverinfo='text'
               )

axis=dict(showbackground=False,
          showspikes=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )

layout = go.Layout(
         #title="%s through Prunned Cifar10 CNN"%target_class,
         title = target_class,
         width=1000,
         height=1000,
         #showlegend=False,
         scene=dict(
             xaxis=dict(axis),
             yaxis=dict(axis),
             zaxis=dict(axis),
        ),
     margin=dict(
        t=100
    ),
    #hovermode='closest',
   )

data=[trace1, trace2]
#data = [trace2]
fig=go.Figure(data=data, layout=layout)



In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash()
app.layout = html.Div([
    dcc.Graph(figure=fig)
])

app.run_server(port=8051)

#py.iplot(fig, filename='small_net')



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8051/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Apr/2020 13:09:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 13:09:15] "GET /_dash-component-suites/dash_renderer/react@16.v1_2_2m1580842230.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 13:09:15] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_2_2m1580842230.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 13:09:15] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_2_2m1580842230.8.6.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 13:09:15] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_2_2m1580842230.7.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 13:09:15] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_8_1m1582848776.min.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 13:09:15] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_8_1m1582848776.js HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2020 13:09